# Lösen von Rekurrenzgleichungen

Viele Algorithmen basieren auf Rekursion. Dann folgt der Zeitaufwand der rekursiven Berechnungsvorschrift des gesuchten Resultats. Für die Effizienzanalyse solcher Algorithmen benötigen wir ein Verfahren, mit dem rekursive Gleichungen (Rekurrenzgleichungen) gelöst werden können.

Ein typisches Beispiel ist die Fibonacci-Folge: $0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55,\ldots$.

Die Bildungsvorschrift der $n$-ten Fibonacci-Zahl lässt sich sehr leicht angegeben, wenn dies rekursiv geschieht:

$$
\begin{align}
fib(0) & = 0 \\
fib(1) & = 1 \\
fib(n) & = fib(n - 1) + fib(n - 2) \mid n \geqslant 2
\end{align}
$$

Die $n$-te Fibonacci-Zahl ist gleich der Summe der zwei vorhergehenden Zahlen.

In [1]:
def fib(n):
    if n == 0:
        return 0
    if n == 1:
        return 1
    return fib(n-1) + fib(n-2)

print(list(map(fib, list(range(0, 20)))))

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181]


Der Zeitaufwand lässt sich ebenfalls durch eine rekursive Gleichung ausdrücken:

$$
\begin{align}
T(0) & = 0 \\
T(1) & = 1 \\
T(n) & = T(n - 1) + T(n - 2), n\geqslant 2
\end{align}
$$

Für $T$ benötigen wir einen Ausdruck in $n$ allerdings ohne $T(i)$ auf der rechten Seite. Gibt es ein Verfahren, um aus der (leicht erkennbaren) rekursiven Definition eine explizite zu gewinnen.

In der Tat gibt es sogar mehrere solcher Verfahren, die das Gewünschte mehr oder weniger erfolgreich leisten.

## Raten und Einsetzen

Eine solche Lösungsmethode ist das __Intelligent guesswork__ - das geschickte Raten. Hierfür stellt man eine Wertetabelle für $T(n)$ auf und versucht daraus eine explizite Bildungsvorschrift zu erkennen.

__Beispiel__

$$
\begin{align}
T(1) & = 1 \\
T(n) & = 3 \cdot T \left(\frac{n}{2} \right) + n
\end{align}
$$

$n$ sei hierbei eine Zweierpotenz, d.h. $n = 2^k$ mit $k \in \mathbb{N}$.

Für die Implementierung mit Python verwenden wir die pandas-Bibliothek zur Verwaltung und Analyse von Daten.

In [2]:
import pandas as pd

def T(n):
    if n == 1:
        return 1
    return 3 * T(n/2) + n


args = list(map(lambda n: 2**n, list(range(1, 8))))

print(pd.DataFrame({'T(n)': pd.Series(map(T, args), index=args, dtype=int)}))

     T(n)
2       5
4      19
8      65
16    211
32    665
64   2059
128  6305


Gibt man zusätzlich zu den Funktionswerten die Summendarstellungen an, ergibt sich folgende Wertetabelle:

|$n$|$T(n)$|
|---:|---:|
|$1$|$1$|
|$2$|$5=3 \cdot 1 + 2$|
|$4$|$19=3^2 \cdot 1 + 3 \cdot 2 + 2^2$|
|$8$|$65=3^3 \cdot 1 + 3^2 \cdot 2 + 3 \cdot 2^2 + 2^3$|
|$16$|$211=3^4 \cdot 1 + 3^3 \cdot 2 + 3^2 \cdot 2^2 + 3 \cdot 2^3 + 2^4$|
|$32$|$665=3^5 \cdot 1 + 3^4 \cdot 2 + 3^3 \cdot 2^2 + 3^2 \cdot 2^3 + 3 \cdot 2^4 + 2^5$|

Mit Hilfe dieser Summendarstellung lässt sich ein gewisses Muster erkennen, dadurch kann die Lösung "erraten" werden.

$$
\begin{align*}
T(2^k) & = 3^k \cdot 2^0 + 3^{k-1} \cdot 2^1 + ... + 3^1 \cdot 2^{k-1} + 3^0 \cdot 2^k \\
 & = \sum_{i=0}^{k}(3^{k-i} \cdot 2^i) \\
 & = 3^k \sum_{i=0}^k \left(\frac{2}{3} \right)^i \\
 & = 3^k \frac{1- \left(\frac{2}{3}^{k+1} \right)}{1-\frac{2}{3}} \\
T(2^k) & = 3^{k+1} - 2^{k+1}
\end{align*}
$$

Um eine Funktion $n \mapsto T(n)$ zu erhalten, muss $k$ durch $\log_2 n$ ersetzt werden.

$$
\begin{align*}
T(n) & = 3^{\log_2 n + 1} - 2^{\log_2 n + 1} \\
 & = 3^{\log_2 n} \cdot 3^1 - 2^{\log_2 n} \cdot 2^1 \\
 & = 3 \cdot 3^{\log_2 n} - 2 \cdot 2^{\log_2 n} \\
T(n) & = 3 \cdot n^{\log_2 3} - 2 \cdot n
\end{align*}
$$

Um die asymptotische Aufwandsordnung anzugeben, können der Summand $-2n$ und der Faktor $3$ vernachlässigt werden. Dies ergibt $\mathcal{O}(n^{\log_2 3})$.

## Iterationsmethode

Bei der __Iterationsmethode__ wird eine rekursive Vorschrift solange angewandt, bis man zu einem rekursionsfreien Ausdruck gelangt. Dies geschieht durch wiederholtes Einsetzen der rekursiven Funktionsaufrufe. Diese Expansion durch Selbstanwendung wird __Telescoping__ genannt.

Hat man eine rekursive Funktion $n\mapsto T(n)$ und setzt man für $n$ einen konkreten Wert ein, so kann problemlos Telescoping angewandt werden, da in endlich vielen Schritten die Elementarfälle erreicht werden und ein rekursionsfreier Ausdruck entsteht. Möchte man aber eine rekursive Funktion $n\mapsto T(n)$ für ein allgemeines $n$ mit Hilfe der Iterationsmethode lösen, so ist ein mathematischer Zwischenschritt nötig.

__Beispiel__

$$
\begin{align*}
T(1) & = 1 \\
T(n) & = 2 \cdot T \left(\frac{n}{4} \right) + n
\end{align*}
$$

Die Gleichung wird nun schrittweise expandiert:

$$
\begin{align*}
T(n) & = 2 \cdot T \left(\frac{n}{4} \right) + n \\
 & = 2 \cdot \left(2 \cdot T \left(\frac{n}{16} \right) + \frac{n}{4} \right) + n \\
 & = 4 \cdot T \left(\frac{n}{16} \right) + \frac{3}{2}n \\
 & = 4 \cdot \left(2 \cdot T \left(\frac{n}{64} \right) + \frac{n}{16} \right) + \frac{3}{2}n \\
 & = 8 \cdot T \left(\frac{n}{64} \right) + \frac{7}{4}n \\
 & = 8 \cdot \left(2 \cdot T \left(\frac{n}{256} \right) + \frac{n}{64} \right) + \frac{7}{4}n \\
 & = 16 \cdot T \left(\frac{n}{256} \right) + \frac{15}{8}n \\
\end{align*}
$$

Es wird ein gewisses Muster für den gesuchten, $T(n)$ definierenden Ausdruck ersichtlich, welches sich mit einer Variable $i$ mit $i\geqslant 1$ ausdrücken lässt.

$$
\begin{align*}
T(n) & = 2^i \cdot T \left(\frac{n}{4^i} \right) + \frac{2^i - 1}{2^{i-1}} \cdot n \\
\end{align*}
$$

Nun muss $i$ so gewählt werden, dass aus $T\left(\frac{n}{4^i}\right)$ ein rekursionsfreier Ausdruck ensteht, d.h. der Elementarfall erreicht ist. Dies geschieht mit $i = \log_4 n$ bei $T(1)$:

$$
\begin{align*}
T\left(\frac{n}{4^i}\right) & = T(1) \\
\frac{n}{4^i} & = 1 \\
n & = 4^i \\
i & = \log_4 n
\end{align*}
$$

Wir setzen $\log_4 n$ für $i$ in dem oben für $T(n)$ angegebenen "Musterausdruck" ein:

$$
\begin{align*}
T(n) & = 2^{\log_4 n} \cdot T \left(\frac{n}{4^{\log_4 n}} \right) + \frac{2^{\log_4 n} - 1}{2^{\log_4 n - 1}} \cdot n \\
 & = n^{\log_4 2} \cdot T(1) + \frac{n^{\log_4 2} - 1}{\frac{2^{\log_4 n}}{2}} \cdot n \\
 & = n^{\log_4 2} \cdot T(1) + \frac{n^{\log_4 2} - 1}{\frac{n^{\log_4 2}}{2}} \cdot n \\
 & = n^{\frac{1}{2}} \cdot T(1) + \frac{n^{\frac{1}{2}} - 1}{\frac{n^{\frac{1}{2}}}{2}} \cdot n \\
 & = n^{\frac{1}{2}} + \frac{2n^{\frac{1}{2}} - 2}{n^{\frac{1}{2}}} \cdot n \\
 & = n^{\frac{1}{2}} + \frac{2n^{\frac{3}{2}} - 2n}{n^{\frac{1}{2}}} \\
 & = n^{\frac{1}{2}} + \frac{2n^{\frac{3}{2}} - 2n}{n^{\frac{1}{2}}} \\
 & = n^{\frac{1}{2}} + 2n - 2n^\frac{1}{2} \\
T(n) & = 2n - n^\frac{1}{2}
\end{align*}
$$

Interessiert man sich nur für die asymptotische Aufwandsordnung, so liegt mit $T(n) \in \mathcal{O}(n)$ ein linearer Zusammenhang vor.

## Meistermethode (Master method)

Die __Meistermethode__ bietet eine Möglichkeit, die asymptotische Aufwandsordnung für [Divide and Conquer-Algorithmen](/notebooks/Documents/algorithmen-und-komplexitaet/08%20-%20Divide%20and%20Conquer.ipynb) anzugeben. Der Zeitaufwand von Divide and Conquer-Algorithmen lässt sich in der Form $T(n) = a \cdot T \left(\frac{n}{b} \right) + f(n)$ angeben. 

__Beispiel__

Für $T(n) = 2 \cdot T \left(\frac{n}{4} \right) + n$ lassen sich $a$, $b$ und $f(n)$ folgendermaßen angeben:
$$
\begin{align}
a & = 2 \\
b & = 4 \\
f(n) & = n
\end{align}
$$

<div class="general-text">
Nun muss man versuchen, den Ausdruck in einen der folgenden drei Fälle einzuordnen. Wenn dies gelingt, ergibt sich die Lösung unmittelbar aus der Variablenbindung. Wenn nicht, ist die Mastermethode zur Lösung der vorliegenden Rekurrenzgleichung nicht anwendbar.
</div>

__Definition 2.1 (Master Theorem)__

### Fall 1

Wenn $f(n) \in \mathcal{O} \left(n^{\log_b a - \epsilon} \right)$ mit $\epsilon > 0$, dann $T(n) \in \Theta \left(n^{\log_b a} \right)$.

Der größte Aufwand besteht hier im Teilen in Subprobleme, die Rekursion ist somit wurzellastig (root-heavy).

### Fall 2

Wenn $f(n) \in \Theta \left(n^{\log_b a} \right)$, dann $T(n) \in \Theta \left(n^{\log_b a}\log n \right)$.

Der Aufwand zum Rekombinieren der gelösten Subprobleme ist gleichwertig mit dem des Teilens.

### Fall 3

Wenn $f(n) \in \Omega \left(n^{\log_b a + \epsilon} \right)$ mit $\epsilon > 0$, dann $T(n) \in \Theta(f(n))$.

In diesem Fall liegt der größte Aufwand im Rekombinieren, die Rekursion ist also blattlastig (leaf-heavy).

__Beispiel__

Für das oben angegebene Beispiel gilt Fall 3 des Master Theorems:

$f(n) = n \in \Omega \left(n^{\log_b a + \epsilon} \right) \implies f(n) \in \Omega \left(n^{\log_4 2 + \epsilon} \right) \implies f(n) \in \Omega \left(n^{\frac{1}{2} + \epsilon} \right)$ mit $\epsilon=\frac{1}{2} > 0$.

Folglich gilt für die Aufwandsordnung $T(n)\in\Theta(f(n)) \implies T(n)\in\Theta(n)$.